In [1]:
import shapely
from shapely import Polygon

from osgeo import gdal
import rasterio

In [2]:
from rasterio.features import rasterize
from rasterstats import zonal_stats

In [3]:
import atlite
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

In [4]:
from scipy.constants import physical_constants

In [5]:
weather_excel_path = "Parameters/weather_parameters.xlsx"

weather_parameters = pd.read_excel(weather_excel_path,
                                       index_col = 'Parameters'
                                       ).squeeze('columns')
weather_filename = weather_parameters['Filename']

hexagons = gpd.read_file('Resources/hex_transport.geojson')

cutout = atlite.Cutout('Cutouts/' + weather_filename +'.nc')
layout = cutout.uniform_layout()

In [ ]:
hexagons.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World: Afghanistan, Albania, Algeria, American Samoa, Andorra, Angola, Anguilla, Antarctica, Antigua and Barbuda, Argentina, Armenia, Aruba, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belgium, Belgium, Belize, Benin, Bermuda, Bhutan, Bolivia, Bonaire, Saint Eustasius and Saba, Bosnia and Herzegovina, Botswana, Bouvet Island, Brazil, British Indian Ocean Territory, British Virgin Islands, Brunei Darussalam, Bulgaria, Burkina Faso, Burundi, Cambodia, Cameroon, Canada, Cape Verde, Cayman Islands, Central African Republic, Chad, Chile, China, Christmas Island, Cocos (Keeling) Islands, Comoros, Congo, Cook Islands, Costa Rica, Côte d'Ivoire (Ivory Coast), Croatia, Cuba, Curacao, Cyprus, Czechia, Denmark, Djibouti, Dominica, Dominican Republic, East Timor, Ecuador, Egypt, El Salvador, Equatoria

In [190]:
pv_profile = cutout.pv(
        panel= 'CSi',
        orientation='latitude_optimal',
        layout = layout,
        shapes = hexagons,
        per_unit = True
        )
pv_profile = pv_profile.rename(dict(dim_0='hexagon'))


[########################################] | 100% Completed | 128.15 s


In [199]:
print(pv_profile.values)
pv_profile.max()

[[0.         0.00957489 0.         ... 0.         0.         0.        ]
 [0.07991671 0.05203675 0.19614189 ... 0.05057594 0.08881711 0.17010495]
 [0.2525509  0.2038703  0.39589965 ... 0.18186935 0.27293826 0.34317198]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


<xarray.DataArray ()> Size: 8B
array(0.81923143)

In [6]:
# hexagons_hydro = gpd.read_file('Data/Laos_inc_hydro_hex.geojson')
# fig, ax = plt.subplots(figsize=(10, 10))
# hexagons_hydro.plot(ax=ax, edgecolor='k', cmap='viridis')
hexagons

,h3_index,n0,n1,n2,n3,n4,n5,waterbody_dist,waterway_dist,road_dist,...,index,theo_turbines,theo_pv,index_right,country,Vientiane road construction costs,Vientiane trucking transport and conversion costs,Vientiane trucking state,Vientiane pipeline transport and conversion costs,geometry
0,854149b3fffffff,284,792,665,218,478,609,0.000000,0.0,0.0,...,0,21.0,254.0,92,Laos,0.0,0.933397,NH3,24.026270,"POLYGON ((102.25839 19.22555, 102.23441 19.317..."
1,854165affffffff,501,187,82,222,102,831,0.000000,0.0,0.0,...,1,190.0,1056.0,92,Laos,0.0,1.047458,NH3,89.138256,"POLYGON ((107.30808 15.46010, 107.28646 15.552..."
2,85659237fffffff,247,794,722,821,739,696,0.000000,0.0,0.0,...,2,239.0,1237.0,92,Laos,0.0,1.018443,NH3,72.574886,"POLYGON ((106.04245 15.34233, 106.02029 15.435..."
3,854060b3fffffff,109,525,728,680,280,937,0.000000,0.0,0.0,...,3,0.0,0.0,92,Laos,0.0,1.024511,NH3,76.038991,"POLYGON ((102.09497 22.22497, 102.07070 22.315..."
4,8564b66bfffffff,101,555,634,193,913,58,0.000000,0.0,0.0,...,4,197.0,905.0,92,Laos,0.0,0.903011,NH3,6.680435,"POLYGON ((102.76235 18.31080, 102.73864 18.403..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,8564900ffffffff,834,775,210,154,0,0,0.000000,0.0,0.0,...,936,67.0,488.0,93,Other,0.0,0.996231,NH3,59.895305,"POLYGON ((100.23869 20.38961, 100.21388 20.481..."
1132,854060bbfffffff,3,680,280,618,0,0,0.000000,0.0,0.0,...,937,2.0,4.0,92,Laos,0.0,1.028981,NH3,78.590945,"POLYGON ((102.14422 22.37783, 102.11995 22.468..."
1133,854060bbfffffff,3,680,280,618,0,0,0.000000,0.0,0.0,...,937,2.0,4.0,139,Other,0.0,1.028981,NH3,78.590945,"POLYGON ((102.14422 22.37783, 102.11995 22.468..."
1134,85649047fffffff,154,832,210,237,824,523,11.131949,0.0,0.0,...,938,67.0,599.0,92,Laos,0.0,0.996388,NH3,59.985236,"POLYGON ((100.45792 20.58299, 100.43317 20.674..."


In [9]:
layout

<xarray.DataArray (y: 35, x: 30)> Size: 4kB
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])
Coordinates:
  * y        (y) float64 280B 14.0 14.25 14.5 14.75 ... 21.75 22.0 22.25 22.5
  * x        (x) float64 240B 100.2 100.5 100.8 101.0 ... 107.0 107.2 107.5

In [10]:
cutout.data

<xarray.Dataset> Size: 886MB
Dimensions:           (x: 30, y: 35, time: 8784)
Coordinates:
  * x                 (x) float64 240B 100.2 100.5 100.8 ... 107.0 107.2 107.5
  * y                 (y) float64 280B 14.0 14.25 14.5 14.75 ... 22.0 22.25 22.5
  * time              (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23...
    lon               (x) float64 240B dask.array<chunksize=(30,), meta=np.ndarray>
    lat               (y) float64 280B dask.array<chunksize=(35,), meta=np.ndarray>
Data variables: (12/13)
    height            (y, x) float64 8kB dask.array<chunksize=(35, 30), meta=np.ndarray>
    wnd100m           (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    roughness         (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_toa        (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_direct     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    ...                ...
    albedo            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_altitude    (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    temperature       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    soil temperature  (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    runoff            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['height', 'runoff', 'wind', 'influx', 'temperature']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2024-04-28 19:27:54 GMT by grib_to_netcdf-2.28.1: /op...

In [11]:
pv_profile.dim_0.data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163], dtype=int64)

In [12]:
pv_profile.sel(dim_0=100).max()

<xarray.DataArray ()> Size: 8B
array(0.75825574)
Coordinates:
    dim_0    int64 8B 100

In [323]:
location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro

,SNo,Status,Fuel Type,lat,lon,capacity,Expected Generation (GWh),total theoretical possible generation (local) GWh,COD,head,geometry
0,1,Existing,Reservoir,18.530336,102.547646,155.0,1025.0,1357.800,1971.0,45.5,POINT (102.54765 18.53034)
1,2,Existing,Run - Off,15.491752,106.278715,45.0,180.0,394.200,1991.0,10.0,POINT (106.27872 15.49175)
2,3,Existing,Reservoir,18.261088,104.562496,440.0,2691.0,0.438,2013.0,27.0,POINT (104.56250 18.26109)
3,4,Existing,Reservoir,15.059603,106.764106,152.0,450.0,17.520,1999.0,79.0,POINT (106.76411 15.05960)
4,5,Existing,Reservoir,18.437522,102.947408,60.0,215.0,525.600,2000.0,45.5,POINT (102.94741 18.43752)
...,...,...,...,...,...,...,...,...,...,...,...
118,197,Expected to be completed after 2030,Reservoir,15.136138,106.593269,81.0,299.0,709.560,NaN,17.0,POINT (106.59327 15.13614)
119,202,Expected to be completed after 2030,Reservoir,20.305807,104.171948,150.0,343.0,1314.000,NaN,71.0,POINT (104.17195 20.30581)
120,250,Expected to be completed after 2030,Reservoir,16.000547,106.961355,330.0,1502.0,2890.800,NaN,133.0,POINT (106.96136 16.00055)
121,266,Expected to be completed after 2030,Run - Off,17.815526,101.538460,660.0,3696.0,87.600,NaN,47.0,POINT (101.53846 17.81553)


In [8]:
hexagons['hydro'] = hexagons['hydro'].fillna(0)
hexagons['hydro'].unique()
# hex_final['capacity'] = hex_final['capacity'].fillna(0)

array([0.000e+00, 4.100e+02, 1.200e+01, 3.600e+01, 3.200e+01, 8.800e+01,
       3.200e+00, 2.600e+02, 1.000e+00, 3.000e+00, 1.450e+01, 2.500e+02,
       1.250e+00, 4.000e+01, 1.800e+02, 5.000e+00, 5.500e+00, 1.500e+01,
       1.400e+01, 1.700e+00, 6.000e+00, 6.600e+01, 1.300e+02, 4.400e+01,
       2.300e+01, 2.100e+02, 6.400e+01, 7.000e+01, 2.400e+02, 2.720e+02,
       6.000e+01, 1.285e+03, 1.080e+03, 1.200e+02, 1.600e+02, 1.000e+02,
       8.000e+00, 9.440e+00, 7.600e+01, 1.920e+01, 1.680e+02, 1.500e+00,
       6.900e+01, 6.500e+02, 6.150e+02, 1.520e+02, 1.320e+02, 1.550e+02])

In [9]:
location_hydro.capacity.max()

1285.0

In [10]:
laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x
laos_hydrobasins.columns

KeyboardInterrupt: 

In [21]:
laos_hydrobasins.to_file('hydrobasins_lvl10/hybas_as_lev10_v1c.gpkg', driver='GPKG')

In [11]:
laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.gpkg')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x
laos_hydrobasins.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'lat', 'lon', 'geometry'],
      dtype='object')

In [12]:
hydro_profile = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

Determine upstream basins per plant: 81it [00:03, 26.81it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 21.32 s


Shift and aggregate runoff by plant: 81it [00:06, 13.39it/s]


In [13]:
hydro_profile

<xarray.DataArray (plant: 81, time: 8784)> Size: 6MB
array([[4790216.34981961, 4768613.92781941, 4745408.08392076, ...,
        4803158.19308756, 4807320.03526345, 4812930.54950411],
       [ 482567.96654921,  483096.88538162,  482898.39702035, ...,
         471429.29190263,  471429.29190263,  471429.29190263],
       [8707366.44057512, 8475747.92012619, 8497575.88942332, ...,
        9138243.02590425, 8953503.7386901 , 8710667.05817353],
       ...,
       [ 139599.88681937,  138499.37368623,  140548.92128137, ...,
         137618.76195345,  137675.95707823,  137789.0172086 ],
       [  35020.89569975,   34553.14688564,   35639.79445784, ...,
          50767.53429779,   48516.62043001,   50767.53429779],
       [  32537.17822724,   32482.22096594,   33145.32420698, ...,
          40388.92326016,   40388.92326016,   40388.92326016]])
Coordinates:
  * plant    (plant) int64 648B 0 1 2 3 4 5 6 7 8 ... 72 73 74 75 76 77 78 79 80
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [14]:
hydro_profile.sel(plant=0).min()

<xarray.DataArray ()> Size: 8B
array(657707.22515643)
Coordinates:
    plant    int64 8B 0

In [15]:
location_hydro.iloc[1,1]

'Existing  '

In [16]:
hydro_profile.dims[1]

'time'

## Match annual hydroprofile with capacity of hydropower
### 1 Equation hydropower potential

In [17]:
location_hydro

,SNo,Status,Fuel Type,lat,lon,capacity,Expected Generation (GWh),total theoretical possible generation (local) GWh,COD,head,geometry
0,1,Existing,Reservoir,18.530336,102.547646,155.0,1025.00,1357.800,1971.0,212.3,POINT (102.54765 18.53034)
1,2,Existing,Run - Off,15.491752,106.278715,45.0,180.00,394.200,1991.0,482.0,POINT (106.27872 15.49175)
2,3,Existing,Reservoir,18.261088,104.562496,440.0,2691.00,0.438,2013.0,400.0,POINT (104.56250 18.26109)
3,4,Existing,Reservoir,15.059603,106.764106,152.0,450.00,17.520,1999.0,883.0,POINT (106.76411 15.05960)
4,5,Existing,Reservoir,18.437522,102.947408,60.0,215.00,525.600,2000.0,406.1,POINT (102.94741 18.43752)
...,...,...,...,...,...,...,...,...,...,...,...
76,89,Existing,Run - Off,19.479770,103.158829,3.0,12.00,26.280,2016.0,55.0,POINT (103.15883 19.47977)
77,91,Existing,Run - Off,19.112470,102.745017,2.5,10.95,21.900,2017.0,209.0,POINT (102.74502 19.11247)
78,92,Existing,Run - Off,19.607903,103.235474,1.0,4.92,8.760,2018.0,254.0,POINT (103.23547 19.60790)
79,95,Existing,Run - Off,18.612302,104.468166,5.0,23.87,43.800,2019.0,0.0,POINT (104.46817 18.61230)


In [18]:
hydro_profile

<xarray.DataArray (plant: 81, time: 8784)> Size: 6MB
array([[4790216.34981961, 4768613.92781941, 4745408.08392076, ...,
        4803158.19308756, 4807320.03526345, 4812930.54950411],
       [ 482567.96654921,  483096.88538162,  482898.39702035, ...,
         471429.29190263,  471429.29190263,  471429.29190263],
       [8707366.44057512, 8475747.92012619, 8497575.88942332, ...,
        9138243.02590425, 8953503.7386901 , 8710667.05817353],
       ...,
       [ 139599.88681937,  138499.37368623,  140548.92128137, ...,
         137618.76195345,  137675.95707823,  137789.0172086 ],
       [  35020.89569975,   34553.14688564,   35639.79445784, ...,
          50767.53429779,   48516.62043001,   50767.53429779],
       [  32537.17822724,   32482.22096594,   33145.32420698, ...,
          40388.92326016,   40388.92326016,   40388.92326016]])
Coordinates:
  * plant    (plant) int64 648B 0 1 2 3 4 5 6 7 8 ... 72 73 74 75 76 77 78 79 80
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [107]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW


eta = 0.75 # Value for system efficiency of hydropower - source
def hydropower_potential_wrapper(flowrate, head):
    return hydropower_potential(eta, flowrate, head)

result = xr.apply_ufunc(
    hydropower_potential_wrapper,
    hydro_profile,
    xr.DataArray(location_hydro['head'], dims=['plant']),  # Ensure head values align with plant dimension
    vectorize=True
)

### 2 Absolute values to capacity factor
#### Change xarray to percentages (If potential > max_capacity: max_capacity ! potential)

In [214]:

# Define the wrapper function to include capacity limiting and capacity factor calculation
def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

# Assuming eta is a constant value, flowrate is an xarray.DataArray, and head is a GeoDataFrame
eta = 0.9  # Example efficiency value
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [24]:
capacity_factor

<xarray.DataArray (plant: 81, time: 8784)> Size: 6MB
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.]])
Coordinates:
  * plant    (plant) int64 648B 0 1 2 3 4 5 6 7 8 ... 72 73 74 75 76 77 78 79 80
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [127]:
location_hydro[location_hydro["SNo"] == 22]

,SNo,Status,Fuel Type,lat,lon,capacity,Expected Generation (GWh),total theoretical possible generation (local) GWh,COD,head,geometry
18,22,Existing,Reservoir,15.356913,106.497986,88.0,480.0,770.88,2015.0,820.0,POINT (106.49799 15.35691)


In [53]:
capacity_factor

<xarray.DataArray (plant: 81, time: 8784)> Size: 6MB
array([[2071.47511028, 2062.13338619, 2052.09828035, ..., 2077.07166465,
        2078.87140642, 2081.29761012],
       [ 473.78370988,  474.30300073,  474.10812548, ...,  462.84779419,
         462.84779419,  462.84779419],
       [7094.49373493, 6905.77810507, 6923.56286152, ..., 7445.55869309,
        7295.03881723, 7097.18297639],
       ...,
       [  72.22596936,   71.65658761,   72.71697931, ...,   71.2009781 ,
          71.23056962,   71.2890645 ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [  20.41292309,   20.37844443,   20.79445701, ...,   25.33889013,
          25.33889013,   25.33889013]])
Coordinates:
  * plant    (plant) int64 648B 0 1 2 3 4 5 6 7 8 ... 72 73 74 75 76 77 78 79 80
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [91]:
# Check if all values for each plant are equal to 1 across the time dimension
all_ones_per_plant = (capacity_factor == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_ones = all_ones_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_ones}")


Number of plants with all values equal to 1: 2


In [90]:
# Check if all values for each plant are equal to 1 across the time dimension
all_ones_per_plant = (capacity_factor == 0).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_ones = all_ones_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 0: {count_all_ones}")


Number of plants with all values equal to 0: 4


In [201]:
location_hydro.at[0,"head"] = 45 # 45.5
location_hydro.at[1,"head"] = 10 # 
location_hydro.at[2,"head"] = 27 # 

In [205]:
location_hydro

,SNo,Status,Fuel Type,lat,lon,capacity,Expected Generation (GWh),total theoretical possible generation (local) GWh,COD,head,geometry
0,1,Existing,Reservoir,18.530336,102.547646,155.0,1025.00,1357.800,1971.0,45.5,POINT (102.54765 18.53034)
1,2,Existing,Run - Off,15.491752,106.278715,45.0,180.00,394.200,1991.0,10.0,POINT (106.27872 15.49175)
2,3,Existing,Reservoir,18.261088,104.562496,440.0,2691.00,0.438,2013.0,27.0,POINT (104.56250 18.26109)
3,4,Existing,Reservoir,15.059603,106.764106,152.0,450.00,17.520,1999.0,79.0,POINT (106.76411 15.05960)
4,5,Existing,Reservoir,18.437522,102.947408,60.0,215.00,525.600,2000.0,45.5,POINT (102.94741 18.43752)
...,...,...,...,...,...,...,...,...,...,...,...
76,89,Existing,Run - Off,19.479770,103.158829,3.0,12.00,26.280,2016.0,24.0,POINT (103.15883 19.47977)
77,91,Existing,Run - Off,19.112470,102.745017,2.5,10.95,21.900,2017.0,54.0,POINT (102.74502 19.11247)
78,92,Existing,Run - Off,19.607903,103.235474,1.0,4.92,8.760,2018.0,42.0,POINT (103.23547 19.60790)
79,95,Existing,Run - Off,18.612302,104.468166,5.0,23.87,43.800,2019.0,51.0,POINT (104.46817 18.61230)


# TESTING 

In [325]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts

    Parameters
    ----------
    eta : float
        Efficiency of the hydropower plant.
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.

    Returns
    -------
    float
        Hydropower potential in Megawatts (MW).
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    '''
    Calculate the hydropower potential considering the capacity limit

    Parameters
    ----------
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.
    capacity : float
        Maximum hydropower capacity in Megawatts (MW).
    eta : float
        Efficiency of the hydropower plant.

    Returns
    -------
    xarray DataArray
        Capacity factor, which is the limited potential divided by the capacity.
    '''
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor


In [326]:
runoff = cutout.hydro(
        plants=location_hydro,
        hydrobasins= laos_hydrobasins,
        per_unit=True                    # Normalize output per unit area
    )

Determine upstream basins per plant: 123it [00:05, 23.98it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 12.34 s


Shift and aggregate runoff by plant: 123it [00:06, 17.98it/s]


In [327]:
x = 75 # index
y = 64 # height dam
# location_hydro.at[x,"head"] = y

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)

print(capacity_factor.sel(plant=x).min())
print(capacity_factor.sel(plant=x).max())
print((capacity_factor.sel(plant=x).sum() * location_hydro.at[x,"capacity"]) / 1000)

<xarray.DataArray ()> Size: 8B
array(1.)
Coordinates:
    plant    int64 8B 75
<xarray.DataArray ()> Size: 8B
array(1.)
Coordinates:
    plant    int64 8B 75
<xarray.DataArray ()> Size: 8B
array(10.98)
Coordinates:
    plant    int64 8B 75


In [374]:
location_hydro[location_hydro["SNo"] == 191] # 79, 

,SNo,Status,Fuel Type,lat,lon,capacity,Expected Generation (GWh),total theoretical possible generation (local) GWh,COD,head,geometry
116,191,Expected to be completed after 2030,Run - Off,19.85034,101.023013,912.0,4846.0,998.64,2032.0,45.0,POINT (101.02301 19.85034)


In [381]:
x = 122 # index
y = 64 # height dam
# location_hydro.at[x,"head"] = y

print(capacity_factor.sel(plant=x).min())
print(capacity_factor.sel(plant=x).max())
print((capacity_factor.sel(plant=x).sum() * location_hydro.at[x,"capacity"]) / 1000)

<xarray.DataArray ()> Size: 8B
array(0.09537822)
Coordinates:
    plant    int64 8B 122
<xarray.DataArray ()> Size: 8B
array(1.)
Coordinates:
    plant    int64 8B 122
<xarray.DataArray ()> Size: 8B
array(1061.45928557)
Coordinates:
    plant    int64 8B 122


In [95]:
print(capacity_factor.sel(plant=1).min())
print(capacity_factor.sel(plant=1).max())
capacity_factor.sel(plant=1).sum() / 1000

<xarray.DataArray ()> Size: 8B
array(1.90469519)
Coordinates:
    plant    int64 8B 1
<xarray.DataArray ()> Size: 8B
array(45.)
Coordinates:
    plant    int64 8B 1


<xarray.DataArray ()> Size: 8B
array(205.40764017)
Coordinates:
    plant    int64 8B 1

### 3 Time-series for each Hexagon
#### Hydro plant capacity factor from plants to hexagons

In [58]:
eta = 0.75
location_hydro.at[0,"head"] = 45.5

In [19]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        # Select the capacity_factor and capacity values for the plants in the current hexagon
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.set_index('plant').loc[plants_in_hex]['capacity'], dims=['plant'])

        # Calculate the weighted average capacity factor for the current hexagon
        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='plant')
        
        # Assign the calculated weighted average to the new DataArray
        capacity_hexagon.loc[hex_index] = weighted_avg_capacity_factor


ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

In [116]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 2


In [117]:
capacity_hexagon

<xarray.DataArray (hexagon: 808, time: 8784)> Size: 57MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 3kB 0 1 2 3 4 5 6 7 ... 801 802 803 804 805 806 807
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [23]:
hexagons

,h3_index,n0,n1,n2,n3,n4,n5,waterbody_dist,waterway_dist,road_dist,index,theo_turbines,theo_pv,index_right,country,Vientiane road construction costs,Vientiane trucking transport and conversion costs,Vientiane trucking state,Vientiane pipeline transport and conversion costs,geometry
0,846496bffffffff,102,93,38,72,89,110,0.0,0.0,0.0,0,1753,9831,92,Laos,0.0,0.535086,NH3,0.447230,"POLYGON ((101.40997 20.06294, 101.43427 20.310..."
1,8440655ffffffff,19,45,33,9,0,0,0.0,0.0,0.0,1,1767,9547,92,Laos,0.0,0.618337,NH3,0.486964,"POLYGON ((102.28949 21.83502, 102.31457 22.078..."
2,84414d3ffffffff,71,6,40,5,10,78,0.0,0.0,0.0,2,1811,9665,92,Laos,0.0,0.437632,NH3,0.400684,"POLYGON ((102.90898 18.78043, 102.93425 19.028..."
3,8440659ffffffff,27,26,19,108,45,0,0.0,0.0,0.0,3,1740,9430,92,Laos,0.0,0.576556,NH3,0.467007,"POLYGON ((102.21447 21.10103, 102.23944 21.346..."
4,8440659ffffffff,27,26,19,108,45,0,0.0,0.0,0.0,3,1740,9430,139,Other,0.0,0.576556,NH3,0.467007,"POLYGON ((102.21447 21.10103, 102.23944 21.346..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,84414e1ffffffff,60,47,96,0,0,0,0.0,0.0,0.0,112,1232,6722,92,Laos,0.0,0.553865,NH3,0.456694,"POLYGON ((104.89164 20.05515, 104.91838 20.300..."
160,8441653ffffffff,103,52,25,65,0,0,0.0,0.0,0.0,113,1324,7132,94,Other,0.0,0.667232,NH3,0.510335,"POLYGON ((107.35758 15.61614, 107.38551 15.864..."
161,8441653ffffffff,103,52,25,65,0,0,0.0,0.0,0.0,113,1324,7132,92,Laos,0.0,0.667232,NH3,0.510335,"POLYGON ((107.35758 15.61614, 107.38551 15.864..."
162,84416c3ffffffff,18,90,74,0,0,0,0.0,0.0,0.0,114,1777,5442,92,Laos,0.0,0.552178,NH3,0.455717,"POLYGON ((105.39455 16.23204, 105.42130 16.481..."


In [31]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)



# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

In [29]:
hydro_hex_mapping

,lat,lon,status,COD,capacity,head,geometry,index_right,h3_index,n0,...,index,theo_turbines,theo_pv,index_right_renamed,country,Vientiane road construction costs,Vientiane trucking transport and conversion costs,Vientiane trucking state,Vientiane pipeline transport and conversion costs,plant_index
0,13.943893,105.955782,COMM,2020,260.000,20,POINT (105.95578 13.94389),156,846590bffffffff,61,...,111,999,4519,90,Other,0.0,0.675338,NH3,0.514323,0
0,13.943893,105.955782,COMM,2020,260.000,20,POINT (105.95578 13.94389),157,846590bffffffff,61,...,111,999,4519,92,Laos,0.0,0.675338,NH3,0.514323,0
1,15.543679,106.260687,COMM,1999,152.100,20,POINT (106.26069 15.54368),97,8465921ffffffff,30,...,69,1825,9267,92,Laos,0.0,0.641315,NH3,0.498153,1
2,15.774865,107.054896,COMM,2023,60.000,20,POINT (107.05490 15.77487),160,8441653ffffffff,103,...,113,1324,7132,94,Other,0.0,0.667232,NH3,0.510335,2
2,15.774865,107.054896,COMM,2023,60.000,20,POINT (107.05490 15.77487),161,8441653ffffffff,103,...,113,1324,7132,92,Laos,0.0,0.667232,NH3,0.510335,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,15.491744,106.278621,COMM,1994,45.000,20,POINT (106.27862 15.49174),97,8465921ffffffff,30,...,69,1825,9267,92,Laos,0.0,0.641315,NH3,0.498153,62
63,15.403775,106.280332,COMM,2009,76.000,20,POINT (106.28033 15.40377),97,8465921ffffffff,30,...,69,1825,9267,92,Laos,0.0,0.641315,NH3,0.498153,63
64,20.346323,104.381276,COMM,2019,9.136,20,POINT (104.38128 20.34632),63,84414e3ffffffff,60,...,47,1772,9575,92,Laos,0.0,0.541980,NH3,0.450516,64
65,19.977092,103.889435,COMM,2019,32.000,20,POINT (103.88944 19.97709),47,844148dffffffff,115,...,35,1785,9700,92,Laos,0.0,0.516568,NH3,0.438627,65


## Combined code

In [78]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    # capacity_factor = limited_potential / capacity
    return limited_potential #capacity_factor



In [63]:
eta = 0.75
location_hydro.at[0,"head"] = 45.5

In [64]:
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [36]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

eta = 0.75 # efficiency of hydropower plant
# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_renamed'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_renamed'})

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.set_index('hexagon').loc[plants_in_hex]['capacity'], dims=['plant'])

        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='hexagon')
        hydro_profile.loc[hex_index] = weighted_avg_capacity_factor

KeyError: "None of ['hexagon'] are in the columns"

In [39]:

location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)

laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x

runoff = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

eta = 0.75 # efficiency of hydropower plant

capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Dask for parallel computation
    output_dtypes=[float]
)

location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)


# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_renamed'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_renamed'})

hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

hydro_profile = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

Determine upstream basins per plant: 67it [00:02, 23.82it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 14.89 s


Shift and aggregate runoff by plant: 67it [00:07,  9.49it/s]


In [66]:
hydro_profile

<xarray.DataArray (hexagon: 931, time: 8784)> Size: 65MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 4kB 0 1 2 3 4 5 6 7 ... 924 925 926 927 928 929 930
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [68]:
grid_profile = xr.full_like(hydro_profile, fill_value=1)
grid_profile

<xarray.DataArray (hexagon: 931, time: 8784)> Size: 65MB
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
Coordinates:
  * hexagon  (hexagon) int32 4kB 0 1 2 3 4 5 6 7 ... 924 925 926 927 928 929 930
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [46]:
hydro_hex_mapping[['plant_index','index_right']]

,plant_index,index_right
0,0,NaN
1,1,888.0
2,2,239.0
3,3,627.0
4,4,485.0
...,...,...
62,62,488.0
63,63,488.0
64,64,422.0
65,65,130.0


In [51]:
location_hydro

,lat,lon,status,COD,capacity,head,geometry
0,13.943893,105.955782,COMM,2020,260.000,20,POINT (105.95578 13.94389)
1,15.543679,106.260687,COMM,1999,152.100,20,POINT (106.26069 15.54368)
2,15.774865,107.054896,COMM,2023,60.000,20,POINT (107.05490 15.77487)
3,15.354431,106.161330,COMM,2021,5.000,20,POINT (106.16133 15.35443)
4,15.356495,106.498949,COMM,2015,84.800,20,POINT (106.49895 15.35650)
...,...,...,...,...,...,...,...
62,15.491744,106.278621,COMM,1994,45.000,20,POINT (106.27862 15.49174)
63,15.403775,106.280332,COMM,2009,76.000,20,POINT (106.28033 15.40377)
64,20.346323,104.381276,COMM,2019,9.136,20,POINT (104.38128 20.34632)
65,19.977092,103.889435,COMM,2019,32.000,20,POINT (103.88944 19.97709)


In [44]:
hydro_hex_mapping.columns

Index(['lat', 'lon', 'status', 'COD', 'capacity', 'head', 'geometry',
       'index_right', 'h3_index', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5',
       'waterbody_dist', 'waterway_dist', 'road_dist', 'hydro', 'index',
       'theo_turbines', 'theo_pv', 'index_right_renamed', 'country',
       'Vientiane road construction costs',
       'Vientiane trucking transport and conversion costs',
       'Vientiane trucking state',
       'Vientiane pipeline transport and conversion costs', 'plant_index'],
      dtype='object')

In [54]:
hexagons = gpd.read_file('Resources/hex_transport.geojson')
hexagons['hydro'] = hexagons['hydro'].fillna(0)

In [62]:
hexagons['Vientiane trucking transport and conversion costs']

0      0.414455
1      0.578949
2      0.673873
3      0.627544
4      0.627544
         ...   
926    0.584484
927    0.531083
928    0.505670
929    0.676506
930    0.669587
Name: Vientiane trucking transport and conversion costs, Length: 931, dtype: float64

In [53]:
for hex_index in range(num_hexagons):
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.loc[plants_in_hex]['capacity'].values, dims=['plant'])

        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='plant')
        hydro_profile.loc[hex_index] = weighted_avg_capacity_factor

In [124]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 0).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 753


## Test PyPsa

In [3]:
from osgeo import gdal
import atlite
import geopandas as gpd
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import p_H2_aux as aux
from functions import CRF
import numpy as np
import logging
import time

import xarray as xr
from scipy.constants import physical_constants

In [4]:
n = pypsa.Network(override_component_attrs=aux.create_override_components())

In [32]:
transport_excel_path = "Parameters/transport_parameters.xlsx"
weather_excel_path = "Parameters/weather_parameters.xlsx"
country_excel_path = 'Parameters/country_parameters.xlsx'
country_parameters = pd.read_excel(country_excel_path,
                                    index_col='Country')
demand_excel_path = 'Parameters/demand_parameters.xlsx'
demand_parameters = pd.read_excel(demand_excel_path,
                                    index_col='Demand center',
                                    ).squeeze("columns")
demand_centers = demand_parameters.index
weather_parameters = pd.read_excel(weather_excel_path,
                                    index_col = 'Parameters'
                                    ).squeeze('columns')
weather_filename = weather_parameters['Filename']

hexagons = gpd.read_file('Resources/hex_transport.geojson')
# !!! change to name of cutout in weather
cutout = atlite.Cutout('Cutouts/' + weather_filename +'.nc')
layout = cutout.uniform_layout()

###############################################################
# Added for hydropower

location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)

laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x

runoff = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

eta = 0.75 # efficiency of hydropower plant

capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Dask for parallel computation
    output_dtypes=[float]
)

location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)


# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_old'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_old'})


hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

hydro_profile = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

for hex_index in range(num_hexagons):
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        average_capacity_factor = hex_capacity_factor.mean(dim='plant')
        hydro_profile.loc[hex_index] = average_capacity_factor

###############################################################

pv_profile = cutout.pv(
    panel= 'CSi',
    orientation='latitude_optimal',
    layout = layout,
    shapes = hexagons,
    per_unit = True
    )
pv_profile = pv_profile.rename(dict(dim_0='hexagon'))

wind_profile = cutout.wind(
    # Changed turbine type - was Vestas_V80_2MW_gridstreamer in first run
    # Other option being explored: NREL_ReferenceTurbine_2020ATB_4MW, Enercon_E126_7500kW
    turbine = 'NREL_ReferenceTurbine_2020ATB_4MW',
    layout = layout,
    shapes = hexagons,
    per_unit = True
    )
wind_profile = wind_profile.rename(dict(dim_0='hexagon'))

Determine upstream basins per plant: 67it [00:02, 24.67it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
INFO:atlite.convert:Convert and aggregate 'runoff'.


[########################################] | 100% Completed | 15.51 s


Shift and aggregate runoff by plant: 67it [00:04, 14.64it/s]
INFO:atlite.convert:Convert and aggregate 'pv'.


[########################################] | 100% Completed | 89.12 s


c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\atlite\resource.py:72: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.
  warnings.warn(msg, FutureWarning)
INFO:atlite.convert:Convert and aggregate 'wind'.


[########################################] | 100% Completed | 26.22 s


In [33]:
def optimize_hydrogen_plant(wind_potential, pv_potential, hydro_potential, times, demand_profile,
                            wind_max_capacity, pv_max_capacity, hydro_max_capacity, 
                            country_series, water_limit = None):
    '''
    Optimizes the size of green hydrogen plant components based on renewable potential, hydrogen demand, and country parameters. 

    Parameters
    ----------
    wind_potential : xarray DataArray
        1D dataarray of per-unit wind potential in hexagon.
    pv_potential : xarray DataArray
        1D dataarray of per-unit solar potential in hexagon.
    times : xarray DataArray
        1D dataarray with timestamps for wind and solar potential.
    demand_profile : pandas DataFrame
        hourly dataframe of hydrogen demand in kg.
    country_series : pandas Series
        interest rate and lifetime information.
    water_limit : float
        annual limit on water available for electrolysis in hexagon, in cubic meters. Default is None.

    Returns
    -------
    lcoh : float
        levelized cost per kg hydrogen.
    wind_capacity: float
        optimal wind capacity in MW.
    solar_capacity: float
        optimal solar capacity in MW.
    electrolyzer_capacity: float
        optimal electrolyzer capacity in MW.
    battery_capacity: float
        optimal battery storage capacity in MW/MWh (1 hour batteries).
    h2_storage: float
        optimal hydrogen storage capacity in MWh.

    '''

    # if a water limit is given, check if hydrogen demand can be met
    if water_limit != None:
        # total hydrogen demand in kg
        total_hydrogen_demand = demand_profile['Demand'].sum()
        # check if hydrogen demand can be met based on hexagon water availability
        water_constraint =  total_hydrogen_demand <= water_limit * 111.57 # kg H2 per cubic meter of water
        if water_constraint == False:
            print('Not enough water to meet hydrogen demand!')
            # return null values
            lcoh = np.nan
            wind_capacity = np.nan
            solar_capacity = np.nan
            hydro_capacity = np.nan
            electrolyzer_capacity = np.nan
            battery_capacity = np.nan
            h2_storage = np.nan
            return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

    # Set up network
    # Import a generic network
    n = pypsa.Network(override_component_attrs=aux.create_override_components())

    # Set the time values for the network
    n.set_snapshots(times)

    # Import the design of the H2 plant into the network
    n.import_from_csv_folder("Parameters/Basic_H2_plant")

    # Import demand profile
    # Note: All flows are in MW or MWh, conversions for hydrogen done using HHVs. Hydrogen HHV = 39.4 MWh/t
    # hydrogen_demand = pd.read_excel(demand_path,index_col = 0) # Excel file in kg hydrogen, convert to MWh
    n.add('Load',
          'Hydrogen demand',
          bus = 'Hydrogen',
          p_set = demand_profile['Demand']/1000*39.4,
          )

    # Send the weather data to the model
    n.generators_t.p_max_pu['Wind'] = wind_potential
    n.generators_t.p_max_pu['Solar'] = pv_potential
    n.generators_t.p_max_pu['Hydro'] = hydro_potential

    # specify maximum capacity based on land use
    n.generators.loc['Wind','p_nom_max'] = wind_max_capacity
    n.generators.loc['Solar','p_nom_max'] = pv_max_capacity 
    n.generators.loc['Hydro','p_nom_max'] = hydro_max_capacity 

    # specify technology-specific and country-specific WACC and lifetime here
    n.generators.loc['Wind','capital_cost'] = n.generators.loc['Wind','capital_cost']\
        * CRF(country_series['Wind interest rate'], country_series['Wind lifetime (years)'])
    n.generators.loc['Solar','capital_cost'] = n.generators.loc['Solar','capital_cost']\
        * CRF(country_series['Solar interest rate'], country_series['Solar lifetime (years)'])
    ### Calculation hydro costs!!! 
    n.generators.loc['Hydro','capital_cost'] = 50
    
    
    for item in [n.links, n.stores,n.storage_units]:
        item.capital_cost = item.capital_cost * CRF(country_series['Plant interest rate'],country_series['Plant lifetime (years)'])

    # Solve the model
    solver = 'gurobi'
    n.lopf(solver_name=solver,
           solver_options = {'LogToConsole':0, 'OutputFlag':0},
           pyomo=False,
           extra_functionality=aux.extra_functionalities,
           )
    # Output results

    lcoh = n.objective/(n.loads_t.p_set.sum()[0]/39.4*1000) # convert back to kg H2
    wind_capacity = n.generators.p_nom_opt['Wind']
    solar_capacity = n.generators.p_nom_opt['Solar']
    hydro_capacity = n.generators.p_nom_opt['Hydro']
    electrolyzer_capacity = n.links.p_nom_opt['Electrolysis']
    battery_capacity = n.storage_units.p_nom_opt['Battery']
    h2_storage = n.stores.e_nom_opt['Compressed H2 Store']
    print(lcoh)
    return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

In [6]:
location = 'Laos'
lcohs_trucking = np.zeros(len(pv_profile.hexagon))
solar_capacities= np.zeros(len(pv_profile.hexagon))
wind_capacities= np.zeros(len(pv_profile.hexagon))
####################################################
# Added for hydropower
hydro_capacites= np.zeros(len(hydro_profile.hexagon))
####################################################
electrolyzer_capacities= np.zeros(len(pv_profile.hexagon))
battery_capacities = np.zeros(len(pv_profile.hexagon))
h2_storages= np.zeros(len(pv_profile.hexagon))
start = time.process_time()
# function
for hexagon in pv_profile.hexagon.data:
    hydrogen_demand_trucking, hydrogen_demand_pipeline = demand_schedule(
        demand_parameters.loc[location,'Annual demand [kg/a]'],
        hexagons.loc[hexagon,f'{location} trucking state'],
        transport_excel_path,
        weather_excel_path)
    country_series = country_parameters.loc[hexagons.country[hexagon]]
    lcoh, wind_capacity, solar_capacity, electrolyzer_capacity, battery_capacity, h2_storage =\
        optimize_hydrogen_plant(wind_profile.sel(hexagon = hexagon),
                            pv_profile.sel(hexagon = hexagon),
                            wind_profile.time,
                            hydrogen_demand_trucking,
                            hexagons.loc[hexagon,'theo_turbines'],
                            hexagons.loc[hexagon,'theo_pv'],
                            country_series,
                            # water_limit = hexagons.loc[hexagon,'delta_water_m3']
                            )
    lcohs_trucking[hexagon] = lcoh
    solar_capacities[hexagon] = solar_capacity
    wind_capacities[hexagon] = wind_capacity
    electrolyzer_capacities[hexagon] = electrolyzer_capacity
    battery_capacities[hexagon] = battery_capacity
    h2_storages[hexagon] = h2_storage

NameError: name 'times' is not defined

In [39]:
1.58E+06 * CRF(0.06, 20)

137751.6000234252

In [41]:
1900000 * CRF(0.06, 20)

165650.65825601766